<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://blog.scholarnest.com/wp-content/uploads/2023/03/scholarnest-academy-scaled.jpg" alt="ScholarNest Academy" style="width: 1400px">
</div>

#####1. Create your bronze layer tables ingesting from the landing zone

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE customers_raw
AS SELECT *, current_timestamp() as load_time
FROM cloud_files('/Volumes/dev/demo_db/landing_zone/customers', 
                 "csv", map("cloudFiles.inferColumnTypes", "true"))

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE invoices_raw
AS SELECT *, current_timestamp() as load_time
FROM cloud_files("/Volumes/dev/demo_db/landing_zone/invoices", 
                 "csv", map("cloudFiles.inferColumnTypes", "true"))

#####2. Create your silver layer tables reading incremental data from bronze layer

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE customers_cleaned (
    CONSTRAINT valid_customer EXPECT (customer_id IS NOT NULL) ON VIOLATION DROP ROW)
AS 
SELECT CustomerID as customer_id, CustomerName as customer_name, load_time   
FROM STREAM(LIVE.customers_raw)

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE invoices_cleaned (
    CONSTRAINT valid_invoice_and_qty EXPECT (invoice_no IS NOT NULL AND quantity > 0) ON VIOLATION DROP ROW)
    PARTITIONED BY (invoice_year, country)
AS
SELECT InvoiceNo as invoice_no, StockCode as stock_code, Description as description,
        Quantity as quantity, to_date(InvoiceDate, "d-M-y H.m") as invoice_date, 
        UnitPrice as unit_price, CustomerID as customer_id, Country as country,
        year(to_date(InvoiceDate, "d-M-y H.m")) as invoice_year, 
        month(to_date(InvoiceDate, "d-M-y H.m")) as invoice_month,
        load_time
FROM STREAM(LIVE.invoices_raw)

#####3. Build your SCD Type 2 dimensions using CDC from silver layer

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE customers;

APPLY CHANGES INTO LIVE.customers
FROM STREAM(LIVE.customers_cleaned)
KEYS (customer_id)
SEQUENCE BY load_time
STORED AS SCD TYPE 2;

#####4. Merge into your fact table using CDC from the silver layer

In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE invoices PARTITIONED BY (invoice_year, country);

APPLY CHANGES INTO LIVE.invoices
FROM STREAM(LIVE.invoices_cleaned)
KEYS (invoice_no, stock_code, invoice_date)
SEQUENCE BY load_time

#####5. Materialize your gold layer summary using silver layer fact

In [0]:
%sql
CREATE LIVE TABLE daily_sales_uk_2022
AS SELECT country, invoice_year, invoice_month, invoice_date,
          round(sum(quantity*unit_price),2) as total_sales
FROM LIVE.invoices
WHERE invoice_year = 2022 AND country="United Kingdom"
GROUP BY country, invoice_year, invoice_month, invoice_date

&copy; 2021-2023 ScholarNest Technologies Pvt. Ltd. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
Databricks, Databricks Cloud and the Databricks logo are trademarks of the <a href="https://www.databricks.com/">Databricks Inc</a>.<br/>
<br/>
<a href="https://www.scholarnest.com/privacy/">Privacy Policy</a> | 
<a href="https://www.scholarnest.com/terms/">Terms of Use</a> | <a href="https://www.scholarnest.com/contact/">Contact Us</a>